## Bridges EDA

In [ ]:
import pandas as pd
import math

In [ ]:
# Read the Bridges file
bridges = pd.read_excel("../data/BMMS_overview.xlsx")

In [ ]:
# Keep only bridges on the N1 road
bridges = bridges.loc[bridges["road"] == "N1"]

# Define bright length bins and labels
length_bins = [0, 10, 50, 200, math.inf]
length_labels = ['S', 'M', 'L', 'XL']

# Categorize bridges per length class
bridges["length_class"] = pd.cut(bridges["length"], bins=length_bins, include_lowest=False, right=False, labels=length_labels)
bridges[["length", "length_class"]].head(10)